In [1]:
from math import*
import numpy as np
from numba import jit,njit
import time
import pandas as pd
import matlab.engine
# Khởi động matlab engine
eng = matlab.engine.start_matlab()
print('MALAB ENGINE FINISHED BEGINNING !!!')

MALAB ENGINE FINISHED BEGINNING !!!


In [2]:
curvature_ref = np.array(pd.read_csv("curvature.csv"))

In [3]:
def init_Nlmpc(x0,Ts):
    x0 = matlab.double(x0)
    Ts = eng.double(0.1)
    nlobj,ekf = eng.InitialNonlinearMPC(x0,Ts, nargout=2)
    return nlobj,ekf

In [65]:
x0 = [0, 0, 15, 0, 0, 0, 0]
u0 = [0, 0, 0.000276764*x0[2], 0.001]
yref = [20, 0, 0]
mv = [0, 0]
md0 = 0.1
white_noise = 0
Ts = 0.1

In [66]:
nlobj, ekf = init_Nlmpc(x0,Ts)


In [68]:
def Nonlinear_MPC_controll(nlmpcObject,ekfObject,x,lastmv,unMeasValue,curvature,Ts,y_reference):
    x= matlab.double(x)
    lastmv = matlab.double(lastmv)
    y_reference = matlab.double(y_reference)
    unMeasValue = eng.double(unMeasValue)
    curvature = matlab.double(curvature)
    Ts = eng.double(Ts)
    output,nlmpcObject,ekfObject = eng.controlByNLMPC(nlmpcObject,ekfObject,x,lastmv,unMeasValue,curvature,Ts,y_reference,nargout = 3)
    output = np.array(output)
    output = [output[0][0],output[1][0]]
    return output,nlmpcObject,ekfObject

In [69]:
curr = curvature_ref[0].tolist()

2257.5605703501024

In [74]:
mv, nlobj, ekf = Nonlinear_MPC_controll(nlobj,ekf,x0,[0,0],0,[0.0044],Ts,yref)

In [75]:
print(mv)

[3.0, 0.055670759232996796]


In [147]:
print(eng.properties(ekf))


['HasAdditiveProcessNoise', 'StateTransitionFcn', 'HasAdditiveMeasurementNoise', 'MeasurementFcn', 'StateTransitionJacobianFcn', 'MeasurementJacobianFcn', 'State', 'StateCovariance', 'ProcessNoise', 'MeasurementNoise']


In [148]:
eng.quit()
print("EXIT MATLAB ENGINE !!!")

EXIT MATLAB ENGINE !!!
